# Sorting out basic design issues

## Running the model for 10 bidders and 10 users

In [69]:
import numpy as np
import random

class User:
    def __init__(self):
        self.probability2 = random.uniform(0, 1)

    def show_ad(self):
        return  np.random.choice((True, False), p = [self.probability2, 1-self.probability2])

class Auction:
    
    auction_history = [] # for the auction exchange only
    balances = {}
    
    def __init__(self, users, bidders):
        self.users = users
        self.bidders = bidders
        
        self.balances = {id(bidder): 0 for bidder in bidders} ########### NEW LINE
        print("*"*100, '\n')
        print('Beginning balances : ',self.balances, '\n','-'*100)
        
        
    def execute_round(self):
        
        ##### if the -ve balance of ALL bidders is < -1000, then break
        
        all_invalid_bidders = any(x > -1000 for x in self.balances.values())
        
        while not all_invalid_bidders:
            print("no bidders are qualifed as all balances are below -1000")
            break
        
        else:
            # print('starting balances2: ', self.balances2, '\n','-'*100)
            bids_dict = {}
            bids_list = []
            bids_list_raw = []

            # 1. SELECT USER (from the pool of users) ----------------------------------

            chosen_user = random.randint(0, len(self.users)-1)
            print('Selected User: ', id(chosen_user), '\n','-'*100) 

            # 2. COLLECT BIDS (send to all bidders) --------------------------------------------
            for bidder in self.bidders:
                #if self.balances[bidder] > -1000:
                #balances = {bidder: 0 for bidder in bidders} ########### NEW LINE
                if self.balances[id(bidder)] > -1000:
                    bids_dict[bidder] = bidder.bid(chosen_user)
                    bidding_round = bidder.bidding_round

                    self.auction_history.append((id(bids_dict[bidder]), bids_dict[bidder])) #bidder
                    highest_bid = 0
                    winning_price = 0
                else:
                    print("Bidder ", id(bidder), "balance is less than -1000. Cannot continue bid" )
     #               #raise Exception("Bidder's balance is less than -1000. Cannot continue bid")
                    continue

            print("Current bidding Round: ", bidding_round, '\n','-'*100)

            # 3. DETERMINE WINNING BID --------------------------------------------
            for bidder, bid_value in bids_dict.items():

                bids_list_raw.append((bidder, bid_value))
                ## only needed to track a human readable bidder id
                bids_list.append((id(bidder), bid_value))


    #        print('bids list', bids_list, '\n','-'*100)

            ###### SORT THE Bids List based on price ##########################################
            sorted_list = sorted(bids_list, key=lambda t:t[1])
            print('Sorted bids list', sorted_list, '\n','-'*100 )

            #------------- for analysis - creating a copy with id(bidder) --------------
            sorted_list_raw = sorted(bids_list_raw, key=lambda t:t[1])
    #        print('sorted bids list RAW', sorted_list_raw, '\n','-'*100 )


            ####### Select second highest price ###############################################
            if len(sorted_list) > 1:
                winning_price = (sorted_list_raw)[-2][1]
                winning_bidder = (sorted_list_raw)[-1][0]
            else:
                winning_price = (sorted_list_raw)[0][-1]
                winning_bidder = (sorted_list_raw)[0][0]

    #        print('Winning Price: ', winning_price, '\n','-'*100)
    #        print('winning bidder', winning_bidder, '\n','-'*100)

            ######## Determine if there are more than 1 bidders with same price ######################

            multiple_winning_bidders = [tup for tup in sorted_list_raw if tup[1] == winning_price]

            if len(multiple_winning_bidders)>1:
                print('List of biders that submitted similar bids: ' , multiple_winning_bidders, '\n','-'*100)
                randindex = random.randint(0, len(multiple_winning_bidders)-1)
                winning_bidder = multiple_winning_bidders[randindex][0]
                print('Winning bidder from list of similar priced bids: ', winning_bidder, '\n','-'*100)
            else: 
                pass
                #winning_bidder = multiple_winning_bidders[0][0]

            print('Winning Bidder (FINAL): ', id(winning_bidder), '\n','-'*100)
            print('Winning Price (FINAL): ', winning_price, '\n','-'*100)

            ############################################################################################
            # 4. Validate USER CLICK after SHOW AD  --------------------------------------------
            ad_result = self.users[chosen_user].show_ad()
            print('User Clicked on Ad?: ', ad_result,'\n','-'*100)

            # 5.NOTIFY BIDDER & UPDATE BALANCES --------------------------------------------

            print("Bidding Outcomes", '\n','-'*100)
            for bidder in self.bidders:
                if bidder == winning_bidder and ad_result == True:
                    print(id(bidder), "bidder == winning_bidder and ad_result == True:")
                    self.balances[id(bidder)] -= winning_price
                    self.balances[id(bidder)] += 1
                    # balances[bidder] -= winning_price
                    # balances[bidder] += 1                

                    bidder.notify(True, winning_price, ad_result)

                elif bidder == winning_bidder and ad_result == False:
                    print(id(bidder), "bidder == winning_bidder and ad_result == False")
                    self.balances[id(bidder)] -= winning_price
                    # balances[bidder] -= winning_price

                    bidder.notify(True, winning_price, ad_result)

                else:
                    print(id(bidder), "Did not win bid - only notify")
                    bidder.notify(False, winning_price, None)
            print('-'*100, '\n','balances at end of bidding round', self.balances,'\n','-'*100)
                # print('balances at end of bidding round', balances,'\n','-'*100)
        
class Bidder:
    
    #### maintain account balance here as well?

    # BIDDER CREATION -------------- initiated by GAME runner / Developer ------------------------
    
    def __init__(self, num_users, num_rounds):

        self.num_users = num_users
        self.num_rounds = num_rounds
        self.bidding_round = 0

        self.bid_y_n = True
        self.bid_participate = {i: 0 for i in range(num_users)} #whether bidder bids or not (USER: BID COUNT)
        self.bid_wins_no_click = {i: 0 for i in range(num_users)} # if bidder wins
        self.bid_win_user_clicks = {i: 0 for i in range(num_users)} # if user clicks
        self.bid_lost = {i: 0 for i in range(num_users)}
        
        #Transaction records
        self.bid_participate_history = [] # maintains bidderid, userid, bidding round, bid price
        self.bid_win_history = [] # won bid
        self.bid_win_user_click_history = [] # won bid and user clicked
        self.bid_lost_history = [] # lost bid, but know the user and price
        self.bid_price = 0
        
        ########### New tables
        self.bid_win_click_no = []
        self.bid_win_click_yes = []
        self.bid_lost_list = []
        self.bid_360 = []
        
        ############
        #? Winning bid price / user (because you get notified of winning bids) -- 
        ###### you know the user and winning bid (just not the click result)
        
    # SUBMIT BID  ------------ initiated by auction ------------------------------------
    
    def bid(self, user_id):
        self.bidding_round += 1
        #self.bid_participate = 0
        self.user_id = user_id
        default_bid_price = random.randint(0, 10000)/200000 #random.uniform(0, 1)
        # bid_price = self.default_bid_price + (self.bidding_round)*0.1
        #self.bid_price = default_bid_price
        self.bid_price = self.bid_strategy(user_id)
        print('bid price from STRATEGY', self.bid_price, '\n','-'*100 )
        #bid_price = 0.5
        self.bid_participate[self.user_id] = (self.bid_participate).get(self.user_id, 0)+1
        self.bid_participate_history.append([id(self), self.user_id, self.bidding_round, self.bid_price])
        return self.bid_price
    
#    def bid_strategy(self):
        
        #self.bid_360.extend(self.bid_win_history, self.bid_lost_history)
        # self.bid_360 = [item for sublst in zip(self.bid_win_history,  self.bid_lost_history) for item in sublst]

        # NOTIFIED OF BID OUTCOME ------------ initiated by auction ------------------------------------
    
    def notify(self, auction_winner, price, clicked):
        bid_y_n_won = 0
        self.auction_winner = auction_winner
        self.price = price
        self.clicked = clicked
        
        if auction_winner == True and clicked == True:
            self.bid_win_user_clicks[self.user_id] = (self.bid_win_user_clicks).get(self.user_id, 0)+1
            #self.bid_win_user_click_history.append([self.user_id, self.bidding_round])
            
            self.bid_win_click_yes.append(['BWCY', self.bidding_round, id(self), self.user_id, self.auction_winner,self.bid_price, self.price, self.clicked])
            
            self.bid_360.append(['BWCY', self.bidding_round, id(self), self.user_id, self.auction_winner,self.bid_price, self.price, self.clicked])

        elif auction_winner == True and clicked == False:
            #self.bid_participate_history.append([self.user_id, bid_y_n_won+1])
            self.bid_wins_no_click[self.user_id] = (self.bid_wins_no_click).get(self.user_id, 0)+1
            # self.bid_win_history.append([self.user_id, bid_y_n_won+1])

#            self.bid_win_history.append([self.bidding_round, self.user_id, self.auction_winner,self.price, self.clicked])
            
            self.bid_win_click_no.append(['BWCN', self.bidding_round, id(self), self.user_id, self.auction_winner,self.bid_price, self.price, self.clicked])
            
            self.bid_360.append(['BWCN', self.bidding_round, id(self), self.user_id, self.auction_winner,self.bid_price, self.price, self.clicked])
        
        else:
            self.bid_lost[self.user_id] = (self.bid_lost).get(self.user_id, 0)+1

#            self.bid_lost_history.append([self.bidding_round, self.user_id, self.auction_winner,self.price, self.clicked])

            self.bid_lost_list.append(['BLOS',self.bidding_round, id(self), self.user_id, self.auction_winner,self.bid_price, self.price, self.clicked])

            self.bid_360.append(['BLOS',self.bidding_round, id(self), self.user_id, self.auction_winner,self.bid_price, self.price, self.clicked])
    def bid_strategy(self, user_id):
#        bidder = self.bidder
        # self.user_id = user_id
        # self.bid_price = bid_price
        self.bid_price = random.randint(0, 10000)/200000 #random.uniform(0, 1)
        self.report = self.bid_360
        # if user_id ==0: #and self.bidder == 'b1':
        #     bid_price = 0.5
        # else: 
        #     bid_price = 1
        return self.bid_price
#        print(self.bid_360)         

In [74]:
b1.bid_strategy(3)

0.04907

In [75]:
dir(b1)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'auction_winner',
 'bid',
 'bid_360',
 'bid_lost',
 'bid_lost_history',
 'bid_lost_list',
 'bid_participate',
 'bid_participate_history',
 'bid_price',
 'bid_strategy',
 'bid_win_click_no',
 'bid_win_click_yes',
 'bid_win_history',
 'bid_win_user_click_history',
 'bid_win_user_clicks',
 'bid_wins_no_click',
 'bid_y_n',
 'bidding_round',
 'clicked',
 'notify',
 'num_rounds',
 'num_users',
 'price',
 'report',
 'user_id']

In [57]:
b1.report

[['BLOS', 1, 140348266201440, 3, False, 0, 0.041055, None],
 ['BLOS', 2, 140348266201440, 9, False, 0, 0.041965, None],
 ['BLOS', 3, 140348266201440, 6, False, 0, 0.044265, None],
 ['BLOS', 4, 140348266201440, 6, False, 0, 0.035725, None],
 ['BLOS', 5, 140348266201440, 7, False, 0, 0.04439, None],
 ['BLOS', 6, 140348266201440, 3, False, 0, 0.044405, None],
 ['BLOS', 7, 140348266201440, 5, False, 0, 0.03607, None],
 ['BWCN', 8, 140348266201440, 4, True, 0, 0.039735, False],
 ['BLOS', 9, 140348266201440, 6, False, 0, 0.045945, None],
 ['BLOS', 10, 140348266201440, 1, False, 0, 0.045045, None],
 ['BLOS', 11, 140348266201440, 5, False, 0, 0.0459, None],
 ['BLOS', 12, 140348266201440, 0, False, 0, 0.026995, None],
 ['BLOS', 13, 140348266201440, 2, False, 0, 0.044655, None],
 ['BLOS', 14, 140348266201440, 9, False, 0, 0.0322, None],
 ['BLOS', 15, 140348266201440, 5, False, 0, 0.042445, None],
 ['BLOS', 16, 140348266201440, 7, False, 0, 0.04699, None],
 ['BLOS', 17, 140348266201440, 6, False,

In [70]:
#b1, b2, b3 = Bidder(1,10), Bidder(1,10), Bidder(1,10)
b1, b2, b3, b4, b5, b6, b7, b8, b9, b10 = Bidder(1,10), Bidder(1,10), Bidder(1,10), Bidder(1,10), Bidder(1,10), Bidder(1,10), Bidder(1,10), Bidder(1,10), Bidder(1,10), Bidder(1,10)
auction = Auction( [User()for i in range(10)], [b1, b2, b3, b4, b5, b6, b7, b8, b9, b10])
#auction = Auction( [User(), User()], [b1, b2, b3])

auction.execute_round()
# auction.execute_round()
# auction.execute_round()
# auction.execute_round()

**************************************************************************************************** 

Beginning balances :  {140348547322496: 0, 140348547322400: 0, 140348547322352: 0, 140348547322304: 0, 140348547322256: 0, 140348547322160: 0, 140348547322112: 0, 140348547322016: 0, 140348547322064: 0, 140348547321920: 0} 
 ----------------------------------------------------------------------------------------------------
Selected User:  140349062310416 
 ----------------------------------------------------------------------------------------------------
bid price from STRATEGY 0.04207 
 ----------------------------------------------------------------------------------------------------
bid price from STRATEGY 0.01896 
 ----------------------------------------------------------------------------------------------------
bid price from STRATEGY 0.04021 
 ----------------------------------------------------------------------------------------------------
bid price from STRATEGY 0.02034

### for b1

In [78]:
# Summary Counts
print('Bidding Rounds', b1.bidding_round)
print('BWCY - Bid Win - Click YES: Count', len(b1.bid_win_click_yes))
print('BWCN - Bid Win - Click NO: Count',len(b1.bid_win_click_no))
print('BLOS - Bid LOST: Count',len(b1.bid_lost_list))
print('Balances', auction.balances)

Bidding Rounds 101
BWCY - Bid Win - Click YES: Count 2
BWCN - Bid Win - Click NO: Count 10
BLOS - Bid LOST: Count 89
Balances {140348547322496: 1.49184, 140348547322400: 0.70697, 140348547322352: 2.3111949999999997, 140348547322304: 2.5246400000000007, 140348547322256: 4.6016200000000005, 140348547322160: 5.520944999999999, 140348547322112: 3.598895, 140348547322016: 2.578485, 140348547322064: 1.6989099999999997, 140348547321920: 0.74095}


## for b2

In [77]:
# Summary Counts
print('Bidding Rounds', b2.bidding_round)
print('BWCY - Bid Win - Click YES: Count', len(b2.bid_win_click_yes))
print('BWCN - Bid Win - Click NO: Count',len(b2.bid_win_click_no))
print('BLOS - Bid LOST: Count',len(b2.bid_lost_list))
print('Balances', auction.balances)

Bidding Rounds 101
BWCY - Bid Win - Click YES: Count 1
BWCN - Bid Win - Click NO: Count 6
BLOS - Bid LOST: Count 94
Balances {140348547322496: 1.49184, 140348547322400: 0.70697, 140348547322352: 2.3111949999999997, 140348547322304: 2.5246400000000007, 140348547322256: 4.6016200000000005, 140348547322160: 5.520944999999999, 140348547322112: 3.598895, 140348547322016: 2.578485, 140348547322064: 1.6989099999999997, 140348547321920: 0.74095}


## for b3

In [79]:
# Summary Counts
print('Bidding Rounds', b3.bidding_round)
print('BWCY - Bid Win - Click YES: Count', len(b3.bid_win_click_yes))
print('BWCN - Bid Win - Click NO: Count',len(b3.bid_win_click_no))
print('BLOS - Bid LOST: Count',len(b3.bid_lost_list))
print('Balances', auction.balances)

Bidding Rounds 101
BWCY - Bid Win - Click YES: Count 3
BWCN - Bid Win - Click NO: Count 13
BLOS - Bid LOST: Count 85
Balances {140348547322496: 1.49184, 140348547322400: 0.70697, 140348547322352: 2.3111949999999997, 140348547322304: 2.5246400000000007, 140348547322256: 4.6016200000000005, 140348547322160: 5.520944999999999, 140348547322112: 3.598895, 140348547322016: 2.578485, 140348547322064: 1.6989099999999997, 140348547321920: 0.74095}


---

In [23]:
print('Bidding Rounds', b1.bidding_round)
print('BWCY - Bid Wins User Clicked', {k: v for k, v in sorted(b1.bid_win_user_clicks.items(), key=lambda item: item[0])})
print('BWCN - Bid Wins User NO Clicked', {k: v for k, v in sorted(b1.bid_wins_no_click.items(), key=lambda item: item[0])})
print('BLOS - Bids Lost', {k: v for k, v in sorted(b1.bid_lost.items(), key=lambda item: item[0])})


Bidding Rounds 101
BWCY - Bid Wins User Clicked {0: 3, 1: 1, 4: 1}
BWCN - Bid Wins User NO Clicked {0: 0, 1: 2, 2: 2, 3: 1, 7: 1, 8: 1}
BLOS - Bids Lost {0: 7, 1: 8, 2: 8, 3: 11, 4: 13, 5: 5, 6: 12, 7: 12, 8: 5, 9: 8}


In [24]:
print('Bid 360: BWCY, BWCN, BLOS: Count', len(b1.bid_360))
b1.bid_360 # 1.Type 2.Round, 3. User, 4. Won Auction? 5.Bid Price 5.Winning Price(2ndBest)  6. User Clicked?

Bid 360: BWCY, BWCN, BLOS: Count 101


[['BLOS', 1, 140349344009088, 2, False, 0, 0.04673, None],
 ['BLOS', 2, 140349344009088, 2, False, 0, 0.04212, None],
 ['BLOS', 3, 140349344009088, 0, False, 0, 0.035635, None],
 ['BLOS', 4, 140349344009088, 4, False, 0, 0.044875, None],
 ['BLOS', 5, 140349344009088, 9, False, 0, 0.031095, None],
 ['BLOS', 6, 140349344009088, 1, False, 0, 0.03932, None],
 ['BLOS', 7, 140349344009088, 8, False, 0, 0.04375, None],
 ['BWCY', 8, 140349344009088, 0, True, 0, 0.0353, True],
 ['BLOS', 9, 140349344009088, 4, False, 0, 0.042615, None],
 ['BLOS', 10, 140349344009088, 5, False, 0, 0.04355, None],
 ['BLOS', 11, 140349344009088, 1, False, 0, 0.0318, None],
 ['BWCN', 12, 140349344009088, 7, True, 0, 0.04604, False],
 ['BLOS', 13, 140349344009088, 7, False, 0, 0.037155, None],
 ['BWCN', 14, 140349344009088, 2, True, 0, 0.03838, False],
 ['BLOS', 15, 140349344009088, 2, False, 0, 0.04685, None],
 ['BLOS', 16, 140349344009088, 4, False, 0, 0.04674, None],
 ['BWCY', 17, 140349344009088, 0, True, 0, 0.04

In [30]:
print('Bid Win - Click YES: Count', len(b1.bid_win_click_yes))
b1.bid_win_click_yes # 1.Type 2.Round, 3. User, 4. Won Auction? 5.Bid Price 5.Winning Price (2ndBest)  6. User Clicked?

Bid Win - Click YES: Count 5


[['BWCY', 4, 140640859103344, 5, True, 0.043035, 0.041445, True],
 ['BWCY', 10, 140640859103344, 1, True, 0.03794, 0.035435, True],
 ['BWCY', 20, 140640859103344, 0, True, 0.04617, 0.04594, True],
 ['BWCY', 46, 140640859103344, 7, True, 0.044385, 0.04417, True],
 ['BWCY', 81, 140640859103344, 7, True, 0.046535, 0.04539, True]]

In [23]:
print('Bid Win - Click NO: Count',len(b1.bid_win_click_no))
b1.bid_win_click_no # 1.Type 2.Round, 3. User, 4. Won Auction? 5.Bid Price 5.Winning Price (2ndBest)  6. User Clicked?

Bid Win - Click NO: Count 138


[['BWCN', 1, 0, True, 1.4435, 1.213, False],
 ['BWCN', 12, 0, True, 0.895, 0.8095, False],
 ['BWCN', 19, 4, True, 1.614, 1.6085, False],
 ['BWCN', 25, 8, True, 1.783, 1.0885, False],
 ['BWCN', 29, 0, True, 1.5705, 1.306, False],
 ['BWCN', 35, 5, True, 1.1455, 1.129, False],
 ['BWCN', 40, 0, True, 1.1875, 0.984, False],
 ['BWCN', 42, 4, True, 1.679, 1.5035, False],
 ['BWCN', 46, 4, True, 1.744, 1.3535, False],
 ['BWCN', 47, 8, True, 1.7755, 1.567, False],
 ['BWCN', 58, 0, True, 1.438, 1.271, False],
 ['BWCN', 60, 0, True, 1.645, 0.473, False],
 ['BWCN', 62, 1, True, 1.6865, 1.4775, False],
 ['BWCN', 67, 4, True, 1.775, 1.6105, False],
 ['BWCN', 78, 0, True, 1.5785, 0.8825, False],
 ['BWCN', 90, 7, True, 1.88, 1.425, False],
 ['BWCN', 110, 9, True, 1.55, 1.512, False],
 ['BWCN', 111, 5, True, 1.817, 0.666, False],
 ['BWCN', 116, 3, True, 0.7625, 0.4175, False],
 ['BWCN', 124, 0, True, 1.957, 0.637, False],
 ['BWCN', 127, 7, True, 1.946, 1.072, False],
 ['BWCN', 129, 4, True, 0.936, 0.84,

In [31]:
print('Bid LOST: Count',len(b1.bid_lost_list))
b1.bid_lost_list # 1.Type 2.Round, 3. User, 4. Won Auction? 5.Bid Price 5.Winning Price (2ndBest)  6. User Clicked?

Bid LOST: Count 92


[['BLOS', 1, 140640859103344, 9, False, 0.03125, 0.041095, None],
 ['BLOS', 2, 140640859103344, 9, False, 0.004675, 0.03882, None],
 ['BLOS', 3, 140640859103344, 5, False, 0.034935, 0.04568, None],
 ['BLOS', 5, 140640859103344, 7, False, 0.033745, 0.045745, None],
 ['BLOS', 6, 140640859103344, 6, False, 0.023025, 0.046865, None],
 ['BLOS', 8, 140640859103344, 4, False, 0.01622, 0.04585, None],
 ['BLOS', 9, 140640859103344, 5, False, 0.04355, 0.048675, None],
 ['BLOS', 11, 140640859103344, 1, False, 0.00711, 0.03592, None],
 ['BLOS', 12, 140640859103344, 5, False, 0.019005, 0.0258, None],
 ['BLOS', 13, 140640859103344, 2, False, 0.028385, 0.04125, None],
 ['BLOS', 14, 140640859103344, 9, False, 0.0373, 0.040565, None],
 ['BLOS', 15, 140640859103344, 0, False, 0.04154, 0.04634, None],
 ['BLOS', 16, 140640859103344, 7, False, 0.01526, 0.044685, None],
 ['BLOS', 17, 140640859103344, 0, False, 0.040365, 0.049525, None],
 ['BLOS', 18, 140640859103344, 0, False, 0.017175, 0.04512, None],
 ['B

In [71]:
[auction.execute_round() for i in range(100)]

Selected User:  140349062310160 
 ----------------------------------------------------------------------------------------------------
bid price from STRATEGY 0.01613 
 ----------------------------------------------------------------------------------------------------
bid price from STRATEGY 0.047175 
 ----------------------------------------------------------------------------------------------------
bid price from STRATEGY 0.027205 
 ----------------------------------------------------------------------------------------------------
bid price from STRATEGY 0.01681 
 ----------------------------------------------------------------------------------------------------
bid price from STRATEGY 0.012405 
 ----------------------------------------------------------------------------------------------------
bid price from STRATEGY 0.014545 
 ----------------------------------------------------------------------------------------------------
bid price from STRATEGY 0.021385 
 ----------------

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [30]:
auction.balances

{140348802358432: 2.5309250000000003,
 140348802359104: 5.627804999999999,
 140348802358672: 9.163195000000002,
 140348802358912: 3.4372000000000003,
 140348802357760: 5.74368,
 140348802358960: 1.80164,
 140348802282304: 4.550605,
 140348802283408: 4.655135,
 140348802283840: 5.495445000000001,
 140348802284656: -0.21222}

In [7]:
b1.bidding_round

101

In [14]:
b1.bid_participate_history #bidderid, userid, bidding round, bid price

[[140467995445472, 5, 1, 2.633]]

In [ ]:
b2.bid_participate_history #bidderid, userid, bidding round, bid price

In [ ]:
b3.bid_participate_history #bidderid, userid, bidding round, bid price

In [ ]:
[auction.execute_round() for i in range(10)]

In [ ]:
b1, b2, b3 = Bidder(1,10), Bidder(1,10), Bidder(1,10)
auction = Auction( [User()], [b1, b2, b3])
auction.execute_round()

In [ ]:
print('b1 bid is: ', b1.bid(0x7fb6581c8ca0))
print('b2 bid is: ', b2.bid(0x7fb6581c8ca0))
print('b3 bid is: ', b3.bid(0x7fb6581c8ca0))

In [ ]:
u1 = User()
id(u1)

In [ ]:
user_wins = {i: 0 for i in range(10)}
user_wins

In [17]:
bids_list = [(1,2), (7,8), (5,6),(3,4) ]

In [18]:
sorted_list = sorted(bids_list, key=lambda t:t[1])
sorted_list

[(1, 2), (3, 4), (5, 6), (7, 8)]

In [19]:
winning_price = (sorted_list)[-2][1]
winning_price

6

In [21]:
winning_bid = (sorted_list)[-1][0]
winning_bid

7

In [22]:
list2 = [(1,2)]
sorted_list = sorted(list2, key=lambda t:t[1])
sorted_list

[(1, 2)]

In [23]:
winning_price = (sorted_list)[0][-1]

In [24]:
winning_price

2

In [26]:
winning_bid = (sorted_list)[0][0]
winning_bid

1